In [1]:

import matplotlib.pyplot as plt
import torch
import torch.nn as nn

from torch.utils.data import DataLoader

from model_init import *
from dataset_init import *
from utils.others import *
from utils.testModel import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Current Device " , device)

batch_size = 64
num_epochs = 1000
num_users = 6
nb_classes = 10
dataset = 'MNIST'

Current Device  cuda


In [3]:
clean_dataset, unlearning_dataset, user_wm_dataset, test_dataset = unlearning_dataset_init(dataset, 6)
print(len(user_wm_dataset))

6


In [4]:
train_loader = DataLoader(unlearning_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [5]:
D, G = model_init(dataset, device)
D.load_state_dict(torch.load('./D-CL.pth'))
optimizerD = torch.optim.Adam(filter(lambda p: p.requires_grad, D.parameters()), 0.0002, betas=(0.5, 0.999))
optimizerG = torch.optim.Adam(G.parameters(), 0.0002, betas=(0.5, 0.999))
criterion_adv = nn.BCELoss()
criterion_aux = nn.CrossEntropyLoss() 

In [6]:
total_step = len(train_loader)
comprehensive_user_test(D, device, test_loader, user_wm_dataset)
for epoch in range(num_epochs):
    for batch_idx, (images, target) in enumerate(train_loader):
        images, target = images.to(device), torch.LongTensor(target).to(device)
        predictR, predictRLabel = D(images) #image from the real dataset
        loss_real_aux = criterion_aux(predictRLabel, target)
        optimizerD.zero_grad()
        optimizerG.zero_grad()
        loss_real_aux.backward()
        optimizerD.step()
        real_score = predictR
    print("epoch",epoch)
    comprehensive_user_test(D, device, test_loader, user_wm_dataset)


Testing on Users' datasset
Testing on User 1 watermark:

Test set: Average loss: 0.0454, Accuracy: 1663/1676 (99%)

Testing on User 2 watermark:

Test set: Average loss: 0.0665, Accuracy: 986/1000 (99%)

Testing on User 3 watermark:

Test set: Average loss: 0.0010, Accuracy: 999/1000 (100%)

Testing on User 4 watermark:

Test set: Average loss: 0.0016, Accuracy: 999/1000 (100%)

Testing on User 5 watermark:

Test set: Average loss: 0.0000, Accuracy: 2500/2500 (100%)

Testing on User 6 watermark:

Test set: Average loss: 0.0078, Accuracy: 1680/1683 (100%)

Testing on clean test set

Test set: Average loss: 0.9705, Accuracy: 7695/10000 (77%)

epoch 0
Testing on Users' datasset
Testing on User 1 watermark:

Test set: Average loss: 10.2049, Accuracy: 29/1676 (2%)

Testing on User 2 watermark:

Test set: Average loss: 7.5456, Accuracy: 52/1000 (5%)

Testing on User 3 watermark:

Test set: Average loss: 16.9553, Accuracy: 0/1000 (0%)

Testing on User 4 watermark:

Test set: Average loss: 5.8

KeyboardInterrupt: 